<a href="https://colab.research.google.com/github/Choa-Lee/UNIUNI_project/blob/master/Action_Classificaion_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abnormal action classification using LSTM
## Last revised by cho : 20.07.25

In [ ]:
%cd /content/drive/My Drive/UNI_project/data

/content/drive/My Drive/UNI_project/data


In [ ]:
!pwd

/content/drive/My Drive/UNI_project/data


## Setting Environment

In [ ]:
import os
import glob
import random
import shutil
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import cv2
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM, Dropout, Activation

In [ ]:
path = '/content/drive/My Drive/UNI_project/data'
all_data_path = path+'/img'
normal_data_path = path+'/label_normal'
abnormal1_data_path = path+'/label_abnormal1'
abnormal2_data_path = path+'/label_abnormal2'

Class = ['normal', 'abnormal1', 'abnormal2']
nClasses = len(Class)

In [ ]:
file_list = glob.glob(all_data_path+'/*')
file_list_jpg = [file for file in file_list if file.endswith(".jpg")]

normal_list = glob.glob(normal_data_path+'/*')
normal_list_jpg = [file for file in normal_list if file.endswith(".jpg")]

abnormal1_list = glob.glob(abnormal1_data_path+'/*')
abnormal1_list_jpg = [file for file in abnormal1_list if file.endswith(".jpg")]

abnormal2_list = glob.glob(abnormal2_data_path+'/*')
abnormal2_list_jpg = [file for file in abnormal2_list if file.endswith(".jpg")]

In [ ]:
print(len(file_list_jpg))
print(len(normal_list_jpg))
print(len(abnormal1_list_jpg))
print(len(abnormal2_list_jpg))

print(len(normal_list_jpg) + len(abnormal1_list_jpg) + len(abnormal2_list_jpg))

11320
5488
2385
3451
11324


In [ ]:
# If you want to copy data from each directories, run this section
# or skip it.

nfiles=min(len(normal_list_jpg), len(abnormal1_list_jpg), len(abnormal2_list_jpg))
min_index = np.argmin([len(normal_list_jpg), len(abnormal1_list_jpg), len(abnormal2_list_jpg)])

if nfiles/2 != 0:
  nfiles = int(nfiles-1)

print(nfiles)

if min_index == 0:
  source_dir=normal_list_jpg
  source_dir1=abnormal1_data_path
  source_dir2=abnormal2_data_path
elif min_index == 1:
  source_dir=abnormal1_list_jpg
  source_dir1=normal_data_path
  source_dir2=abnormal2_data_path
else:
  source_dir=abnormal2_list_jpg
  source_dir1=normal_data_path
  source_dir2=abnormal1_data_path

print(source_dir[0])
print(source_dir1)
print(source_dir2)

dest_dir = path+'/dest_dir'
!mkdir dest_dir


random_file_list1 = []

while len(random_file_list1) < int(nfiles/2):
  random_file1=random.choice(os.listdir(source_dir1))

  fileExt=os.path.splitext(random_file1)[-1]
  if fileExt == '.jpg':
    random_file_list1.append(random_file1)
    random_file_list1 = list(set(random_file_list1))

print(len(random_file_list1))

random_file_list2 = []

while len(random_file_list2) < int(nfiles/2):
  random_file2=random.choice(os.listdir(source_dir2))

  fileExt=os.path.splitext(random_file2)[-1]
  if fileExt == '.jpg':
    random_file_list2.append(random_file2)
    random_file_list2 = list(set(random_file_list2))

print(len(random_file_list2))

for i in range(int(nfiles/2)):
  source_file = os.path.join(source_dir[i])
  source_file1 = os.path.join(source_dir1, random_file_list1[i])
  source_file2 = os.path.join(source_dir2, random_file_list2[i])

  shutil.copy(source_file, dest_dir)
  shutil.copy(source_file1, dest_dir)
  shutil.copy(source_file2, dest_dir)

2384


# Load Data

In [ ]:
data = []; labels = []
nfiles=2381

for image_path in normal_list_jpg:
  while len(labels) < int(nfiles):
    image = cv2.imread(image_path)
    image/255
    image=image.reshape(image.shape[0]*image.shape[1],image.shape[2])
    label = Class[0]

    data.append(image)
    labels.append(label)

print(len(data))
print(len(labels))

for image_path in abnormal1_list_jpg:
  while len(labels) < int(nfiles*2):
    image = cv2.imread(image_path)
    image/255
    image=image.reshape(image.shape[0]*image.shape[1],image.shape[2])
    label = Class[1]

    data.append(image)
    labels.append(label)

print(len(data))
print(len(labels))

for image_path in abnormal2_list_jpg:
  while len(labels) < int(nfiles*3):
    image = cv2.imread(image_path)
    image/255
    image=image.reshape(image.shape[0]*image.shape[1],image.shape[2])
    label = Class[2]

    data.append(image)
    labels.append(label)

print(len(data))
print(len(labels))


2381
2381
4762
4762
7143
7143


In [ ]:
print(len(data))
print(len(labels))
print(data[0].shape)

7143
7143
(4800, 3)


In [ ]:
data = np.array(data)
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

#LSTM model building

## Parameters setting

In [ ]:
test =0.25
learning_rate = 0.001
epoch = 10
batch = 200

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=test, 
                                                  stratify=labels, random_state=42)


n_unit = 64 #hidden LSTM units
n_Samples = trainX.shape[0]
n_time_steps = trainX.shape[1]
n_features = trainX.shape[2]
# n_output = trainY.shape[1]
input_shape=(n_time_steps, n_features) #(nSamples, time steps, nFeatures >> 1)
batch_input_shape=(n_Samples, n_time_steps, n_features)

In [ ]:
print(trainX.shape)

(5357, 4800, 3)


In [ ]:
model = Sequential()
model.add(LSTM(units=64, input_shape=input_shape, activation='relu', return_sequences=True))
# model.add(LSTM(units=10, input_shape=input_shape, activation='relu', return_sequences=True))
model.add(LSTM(units=10))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
# model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='softmax'))

# model.add(Dense(n_outputs, activation='softmax')
model.summary()

adam = optimizers.Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])#loss='mean_squared_error'


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 4800, 32)          4608      
_________________________________________________________________
lstm_53 (LSTM)               (None, 10)                1720      
_________________________________________________________________
dropout_19 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_66 (Dense)             (None, 3)                 18        
Total params: 6,511
Trainable params: 6,511
Non-trainable params: 0
___________________________________________________

In [ ]:
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(5357, 4800, 3) (1786, 4800, 3) (5357, 3) (1786, 3)


In [ ]:
history_lstm = model.fit(trainX, trainY, epochs=10, batch_size=200, verbose=1)

Epoch 1/10
27/27 [==============================] - 216s 8s/step - loss: nan - accuracy: 0.3541
Epoch 2/10
27/27 [==============================] - 212s 8s/step - loss: nan - accuracy: 0.3332
Epoch 3/10
27/27 [==============================] - 208s 8s/step - loss: nan - accuracy: 0.3332
Epoch 4/10
27/27 [==============================] - 208s 8s/step - loss: nan - accuracy: 0.3332
Epoch 5/10
27/27 [==============================] - 212s 8s/step - loss: nan - accuracy: 0.3332
Epoch 6/10
27/27 [==============================] - 209s 8s/step - loss: nan - accuracy: 0.3332
Epoch 7/10
27/27 [==============================] - 211s 8s/step - loss: nan - accuracy: 0.3332
Epoch 8/10
27/27 [==============================] - 209s 8s/step - loss: nan - accuracy: 0.3332
Epoch 9/10
27/27 [==============================] - 205s 8s/step - loss: nan - accuracy: 0.3332
Epoch 10/10
27/27 [==============================] - 204s 8s/step - loss: nan - accuracy: 0.3332


In [ ]:
model = Sequential()
model.add(LSTM(units=64, input_shape=input_shape, activation='relu', return_sequences=True))
# model.add(LSTM(units=10, input_shape=input_shape, activation='relu', return_sequences=True))
model.add(LSTM(units=10))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
# model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='softmax'))

# model.add(Dense(n_outputs, activation='softmax')
model.summary()

adam = optimizers.Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy'])#loss='mean_squared_error'


Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_54 (LSTM)               (None, 4800, 64)          17408     
_________________________________________________________________
lstm_55 (LSTM)               (None, 10)                3000      
_________________________________________________________________
dropout_20 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_67 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_68 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_69 (Dense)             (None, 3)                 18        
Total params: 20,591
Trainable params: 20,591
Non-trainable params: 0
_________________________________________________

In [ ]:
history_lstm = model.fit(trainX, trainY, epochs=10, batch_size=200, verbose=1)

Epoch 1/10
27/27 [==============================] - 311s 12s/step - loss: nan - accuracy: 0.3332
Epoch 2/10
27/27 [==============================] - 310s 11s/step - loss: nan - accuracy: 0.3332
Epoch 3/10
27/27 [==============================] - 307s 11s/step - loss: nan - accuracy: 0.3332
Epoch 4/10
27/27 [==============================] - 309s 11s/step - loss: nan - accuracy: 0.3332
Epoch 5/10
27/27 [==============================] - 307s 11s/step - loss: nan - accuracy: 0.3332
Epoch 6/10
27/27 [==============================] - 310s 11s/step - loss: nan - accuracy: 0.3332
Epoch 7/10
27/27 [==============================] - 310s 11s/step - loss: nan - accuracy: 0.3332
Epoch 8/10
27/27 [==============================] - 308s 11s/step - loss: nan - accuracy: 0.3332
Epoch 9/10
27/27 [==============================] - 305s 11s/step - loss: nan - accuracy: 0.3332
Epoch 10/10
27/27 [==============================] - 309s 11s/step - loss: nan - accuracy: 0.3332


In [ ]:
scores = model.evaluate(testX, testY, verbose=1)
print('Accuracy: %.2f%%' % (scores[1]*100))

y_pred = model.predict(testX)

size=
Q = deque(maxlen=size)
Q.append(y_pred)
results = np.array(Q).mean
i = np.argmax(results)
label = lb[i]


y_test_ = np.argmax(testY, axis=1)
print(accuracy_score(y_pred, y_test_))

In [ ]:
plt.plot(history_lstm.history['loss'])
plt.title('Loss')
plt.show()